##  Finalising the blockchain

The aim of this work is to create a BlockchainState class which keeps track of the blockchain state  and its users state.
Moreover, we add the function reorg which will take care of the forks that are created, keeping the longest chain the fork which has more work done on it.

Additional changes were made in the class Blocks as part of this work. 

In [1]:
#Import relevant libraries
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
import time
from transactions import *
from blocks import *
import copy


In [2]:
class BlockchainState:
    """
    This class keeps track of the longest chain as well as the user's states and the total difficulty
    """
    def __init__(self, longest_chain, user_states, total_difficulty):
        self.longest_chain=longest_chain
        self.user_states=user_states
        self.total_difficulty=total_difficulty
    
    def calculate_difficulty(self):
        """
        Calculates the difficulty for the next block
        """
        #the difficulty is calculated based on the previous 10 blocks difficulties
        # and the time between the 11th previous block and the previous block
        #hence when the chain lenght is shorter or equal to 10, a small difficulty is used.
        if len(self.longest_chain) <= 10:
            return 1000
        else:
            previous_10=self.longest_chain[-10:]
            total_difficulty_for_period=0
            for block in previous_10:
                total_difficulty_for_period+=block.difficulty
            total_time_for_period=max(self.longest_chain[-1].timestamp - self.longest_chain[-11].timestamp,1)
            return (total_difficulty_for_period // total_time_for_period) * 120
    
    def verify_and_apply_block(self, block):
        """
        this function verifies the block variables and verifies the transactions within the block
        apply changes on the user states and add the block to the chain.
        """
        #check block height is the lenght of the longest_chain
        assert block.height==len(self.longest_chain), "incorrect heigth"
        
        #checks that the previous variable in the block is equal to the previous block_id or to 0, in case it is the first block
        #checks that the block timestamp is equal or higher than the previous timestamp
        if self.longest_chain==[]:
            firts_block=0
            previous=firts_block.to_bytes(32, byteorder = 'little', signed = False)
            assert block.previous== previous, "previous block id"
            
        else:
            assert block.previous == self.longest_chain[-1].block_id, "previous block id"
            
            assert block.timestamp >= self.longest_chain[-1].timestamp, "incorrect timing"
        #calculates the difficulty to be verified
        difficulty=self.calculate_difficulty()
        #verifies the block and the transactions within it and apply changes in the user states
        user_states_new=block.verify_and_get_changes(difficulty, self.user_states)
        
        #apply changes
        self.user_states=user_states_new
        self.longest_chain.append(block)
        self.total_difficulty+=block.difficulty
    
    def undo_last_block(self):
        """
        This functions undo the last block that was added to the blockchain 
        """
        self.total_difficulty=self.total_difficulty - self.longest_chain[-1].difficulty
        self.user_states=self.longest_chain[-1].get_changes_for_undo(self.user_states)
        self.longest_chain=self.longest_chain[:-1]
        

In [3]:
def verify_reorg(old_state, new_branch):
    """
    This function can reorg the blockchain when there is a new brach that has higher difficulty than the previous branch
    making the longest chain, the one that has more work done on it.
    """
    #copy the old state
    new_state=copy.deepcopy(old_state)
    #gets the hight of the initial block in the new brach
    height=new_branch[0].height
    #iterate until the hight of the longest blockchain is equal to the initial block in the new branch
    #and undo each of the blocks in the old state and change back the user states and difficulty
    while(new_state.longest_chain[-1].height>= height):
        new_state.undo_last_block()
    #adds the blocks of the new brach and changes the users states
    for block in new_branch:
        new_state.verify_and_apply_block(block)
        
    #verifies that the new blockchain, with the new brach, has higher difficulty than the old branch    
    assert new_state.total_difficulty> old_state.total_difficulty, "total difficulty"
    
    return new_state

In [4]:
#I start by Creating 10 users
#5 senders start with balance 1000
#5 receivers start with balance 0
#all 10 start with nonce -1
senders_private_keys=[]
senders_address=[]
receivers_address=[]
#I Create a senders and recipient addresses from eliptic curves and hashing
for i in range(10):
    private_key=ec.generate_private_key(ec.SECP256K1)
    public_key_not_encoded = private_key.public_key()
    public_key=public_key_not_encoded.public_bytes(encoding=serialization.Encoding.DER, 
                                                                 format=serialization.PublicFormat.SubjectPublicKeyInfo)
    sha1 = hashes.Hash(hashes.SHA1())
    sha1.update(public_key)
    address=sha1.finalize()
    if i <5:
        senders_private_keys.append(private_key)
        senders_address.append(address)
    else:
        receivers_address.append(address)       
#I Generate user states for receivers and senders        
senders=[]
receivers=[]
for i in range(10):
    if i<5:
        senders.append(generate_new_user(1000,-1))
    else:
        receivers.append(generate_new_user(0,-1))
#I Generate the previous_user_states dictionary
senders_address_states=dict(zip(senders_address,senders))
receivers_address_states=dict(zip(receivers_address,receivers))
previous_user_states = senders_address_states.copy()
for key, value in receivers_address_states.items():
    previous_user_states[key] = value


In [5]:
#I Initiate the Blockchain
Blockchain=BlockchainState(longest_chain=[], user_states=previous_user_states, total_difficulty=0)

In [6]:
#I iterate 12 times to create a blockchain of 12 blocks 
for n in range(12):
    #I Create the transaction list, all senders sends to all receivers
    #All transactions will be done with the same amount and fee
    amount=10
    fee=1
    Transactions=[]
    for i in range(5):
        if n==0:
            nonce=0
        else:
            nonce=Blockchain.user_states[senders_address[i]].nonce + 1
        for j in range(5):
            Tr= create_signed_transaction(senders_private_keys[i], receivers_address[j], amount, fee, nonce)
            Transactions.append(Tr)
            nonce+=1
    #I Create a new block
    #I create a miner address 
    miner_private_key=ec.generate_private_key(ec.SECP256K1)
    miner_public_key_not_encoded = miner_private_key.public_key()
    miner_public_key=miner_public_key_not_encoded.public_bytes(encoding=serialization.Encoding.DER,
                                                               format=serialization.PublicFormat.SubjectPublicKeyInfo)
    miner_sha1 = hashes.Hash(hashes.SHA1())
    miner_sha1.update(miner_public_key)
    miner_address=miner_sha1.finalize()
    #I calculate the difficulty of the block
    difficulty=Blockchain.calculate_difficulty()
    if Blockchain.longest_chain==[]:
        firts_block=0 #the first block uses 0 as its previous block
        previous=firts_block.to_bytes(32, byteorder = 'little', signed = False)
    else:
        #the previous parameter is the latest block id 
        previous=Blockchain.longest_chain[-1].block_id
    if n<10 and n>0:
        time.sleep(120)
        #Mine a block
    #I mine a block every 2 minutes when the difficulty is low, then, the difficulty is calculated accordingly and the next 
    #blocks should be mined every 2 minutes
    block=mine_block(previous=previous, height=n, miner=miner_address, transactions=Transactions, timestamp=int(time.time()), 
                         difficulty=difficulty, cutoff_time= time.time()+60)
    #add the block to the blockchain
    Blockchain.verify_and_apply_block(block)
    print("{} block(s) mined".format(n+1))
    
    #I save the users states and the difficulty at the 9th block so I can create a fork
    if n==8:
        user_states_at_9th_block=copy.deepcopy(Blockchain.user_states)
        total_difficulty_at_9th_block=copy.deepcopy(Blockchain.total_difficulty)

0 block(s) mined
1 block(s) mined
2 block(s) mined
3 block(s) mined
4 block(s) mined
5 block(s) mined
6 block(s) mined
7 block(s) mined
8 block(s) mined
9 block(s) mined
10 block(s) mined
11 block(s) mined


In [7]:
#Create alternative blockchain
#This brack will start from the 9th block of the main blockchain so I copied the blockchain till the 9th block
new_blockchain_alternative=BlockchainState(longest_chain=Blockchain.longest_chain[:9], user_states=user_states_at_9th_block,
                           total_difficulty=total_difficulty_at_9th_block)

#I generate 5 new blocks in the alternative branch
for n in range(5):
    #I Create the transaction list, all senders sends to all receivers
    #All transactions will be done with the same amount and fee
    amount=5
    fee=1
    Transactions=[]
    for i in range(5):
        nonce=new_blockchain_alternative.user_states[senders_address[i]].nonce + 1
        for j in range(5):
            Tr= create_signed_transaction(senders_private_keys[i], receivers_address[j], amount, fee, nonce)
            Transactions.append(Tr)
            nonce+=1
    #I Create a new block
    #I create a miner address 
    miner_private_key=ec.generate_private_key(ec.SECP256K1)
    miner_public_key_not_encoded = miner_private_key.public_key()
    miner_public_key=miner_public_key_not_encoded.public_bytes(encoding=serialization.Encoding.DER,
                                                               format=serialization.PublicFormat.SubjectPublicKeyInfo)
    miner_sha1 = hashes.Hash(hashes.SHA1())
    miner_sha1.update(miner_public_key)
    miner_address=miner_sha1.finalize()

    difficulty=new_blockchain_alternative.calculate_difficulty()
    previous=new_blockchain_alternative.longest_chain[-1].block_id
        #Create block
    if n<=1:
        time.sleep(120)
        #I mine a block every 2 minutes when the difficulty is low, then, the difficulty is calculated accordingly and the next 
        #blocks should be mined every 2 minutes
    block=mine_block(previous=previous, height=9 +n, miner=miner_address, transactions=Transactions, timestamp=int(time.time()), 
                         difficulty=difficulty, cutoff_time= time.time()+60)
    new_blockchain_alternative.verify_and_apply_block(block)
    print("{} block(s) mined".format(n+1))
    
#the new branch will be the list of blocks that I added in the copied blockchain    
new_branch=new_blockchain_alternative.longest_chain[-5:]

0 block(s) mined
1 block(s) mined
2 block(s) mined
3 block(s) mined
4 block(s) mined


In [8]:
#Now the blockchain will be reorg with the new branch 
Blockchain=verify_reorg(Blockchain, new_branch)

In [9]:
#verification of the blochain lenght
len(Blockchain.longest_chain)

14